In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import pandas as pd
import pickle
import random

import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from keras.optimizers import RMSprop, Adam, Adamax, Adagrad

import keras_tuner

from lib.read_data import read_and_join_output_file
#from lib.create_pipeline import create_transformation_pipeline
from lib.deeplearning import create_transformation_pipelines
from lib.transform_impute import convert_back_df
from lib.split_data import train_test_group_time_split

In [3]:
RANDOM_SEED = 31
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tensorflow.random.set_seed(RANDOM_SEED)

In [4]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
# During experiment we can try to use neptune.ai to log all the Tensorflow experiments results
neptune_key = pickle.load(open("./neptune.pkl", "rb"))


## Preparing the Dataset
The train and test sets are split by Township-Ranges, i.e. some Township-Ranges data are either fully in the train or test set.
The target value is the value of that variable for 2021
Thus train/test sets are of shape (number of Township-Ranges, 7 years (2014-2020), the number of features).
The input of 1 data point in the model is of shape (7x81


In [6]:
test_size=0.15
# Load the data from the ETL output files
X = read_and_join_output_file()
#X["WELL_COUNT"] = X["WELL_COUNT_PUBLIC"] + X["WELL_COUNT_AGRICULTURE"] + X["WELL_COUNT_DOMESTIC"] + X["WELL_COUNT_INDUSTRIAL"]
#X.drop(columns=["WELL_COUNT_PUBLIC", "WELL_COUNT_AGRICULTURE", "WELL_COUNT_DOMESTIC", "WELL_COUNT_INDUSTRIAL"], inplace=True)
# Split the data into a training and a test set
X_train_df, X_test_df, y_train_df, y_test_df = train_test_group_time_split(X, index=["TOWNSHIP_RANGE", "YEAR"], group="TOWNSHIP_RANGE", test_size=test_size, random_seed=RANDOM_SEED)
# Create, fit and apply the data imputation pipeline to the training and test sets
impute_pipeline, columns = create_transformation_pipelines(X_train_df)
X_train_impute = impute_pipeline.fit_transform(X_train_df)
X_test_impute = impute_pipeline.transform(X_test_df)
# Convert the X_train and X_test back to dataframes
X_train_impute_df = pd.DataFrame(X_train_impute, index=X_train_df.index, columns=columns)
X_test_impute_df = pd.DataFrame(X_test_impute, index=X_test_df.index, columns=columns)
#X_train_impute_df.drop(columns=["BOTTOMOFPERFORATEDINTERVAL_AVG", "TOPOFPERFORATEDINTERVAL_AVG"], inplace=True)
#X_test_impute_df.drop(columns=["BOTTOMOFPERFORATEDINTERVAL_AVG", "TOPOFPERFORATEDINTERVAL_AVG"], inplace=True)
# Keep only the GSE_GWE variable as the outcome variable
scaler = MinMaxScaler()
y_train = scaler.fit_transform(y_train_df[["GSE_GWE"]])
y_test = scaler.transform(y_test_df[["GSE_GWE"]])
y_train_3d = y_train[..., np.newaxis]
X_train_impute_df

TOTALDRILLDEPTH_AVG  WELLYIELD_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                             
T01N R02E      2014             0.000000       0.000489              0.020868   
               2015             0.000000       0.000000              0.000000   
               2016             0.000000       0.003259              0.036728   
               2017             0.066667       0.006410              0.025876   
               2018             0.053651       0.000652              0.063022   
...                                  ...            ...                   ...   
T32S R30E      2016             0.000000       0.000000              0.000000   
               2017             0.000000       0.000000              0.000000   
               2018             0.000000       0.000000              0.000000   
               2019             0.000000       0.000000              0.000000   
               2020             0.000000       0.000000              0.000000   

                     TOPOFPERFORATEDINTERVAL_AVG  \
TOWNSHIP_RANGE YEAR                                
T01N R02E      2014                     0.052288   
               2015                     0.000000   
               2016                     0.084967   
               2017                     0.082789   
               2018                     0.077124   
...                                          ...   
T32S R30E      2016                     0.000000   
               2017                     0.000000   
               2018                     0.000000   
               2019                     0.000000   
               2020                     0.000000   

                     BOTTOMOFPERFORATEDINTERVAL_AVG  TOTALCOMPLETEDDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                            
T01N R02E      2014                        0.076699                 0.127841   
               2015                        0.000000                 0.000000   
               2016                        0.058252                 0.056818   
               2017                        0.064725                 0.074495   
               2018                        0.053592                 0.064015   
...                                             ...                      ...   
T32S R30E      2016                        0.000000                 0.000000   
               2017                        0.000000                 0.000000   
               2018                        0.000000                 0.000000   
               2019                        0.000000                 0.000000   
               2020                        0.000000                 0.000000   

                     VEGETATION_BLUE_OAK-GRAY_PINE  \
TOWNSHIP_RANGE YEAR                                  
T01N R02E      2014                       0.010798   
               2015                       0.010798   
               2016                       0.010798   
               2017                       0.010798   
               2018                       0.010798   
...                                            ...   
T32S R30E      2016                       0.033178   
               2017                       0.033178   
               2018                       0.033178   
               2019                       0.033178   
               2020                       0.033178   

                     VEGETATION_CALIFORNIA_COAST_LIVE_OAK  \
TOWNSHIP_RANGE YEAR                                         
T01N R02E      2014                              0.002749   
               2015                              0.002749   
               2016                              0.002749   
               2017                              0.002749   
               2018                              0.002749   
...                                                   ...   
T32S R30E      2016                              0.000000 

In [7]:
# Change the shape of the input array to (number of Township-Ranges, 7 years (2014-2020), the number of features)
X_train = X_train_impute_df.values.reshape(len(X_train_impute_df.index.get_level_values(0).unique()), len(X_train_impute_df.index.get_level_values(1).unique()), X_train_impute_df.shape[1])
X_test = X_test_impute_df.values.reshape(len(X_test_impute_df.index.get_level_values(0).unique()), len(X_test_impute_df.index.get_level_values(1).unique()), X_test_impute_df.shape[1])

In [8]:
print("="*100)
print("Checking the train, validation and test input (X) datasets sizes:")
print(f"Size of the X_train dataset: {X_train.shape}")
#print(f"Size of the X_val dataset: {X_val.shape}")
print(f"Size of the X_test dataset: {X_test.shape}")
print("="*100)
print("Checking the train, validation and test output (y) datasets sizes:")
print(f"Size of the y_train dataset: {y_train.shape}")
#print(f"Size of the y_val dataset: {y_val_df.shape}")
print(f"Size of the y_test dataset: {y_test.shape}")

Checking the train, validation and test input (X) datasets sizes:
Size of the X_train dataset: (406, 7, 81)
Size of the X_test dataset: (72, 7, 81)
Checking the train, validation and test output (y) datasets sizes:
Size of the y_train dataset: (406, 1)
Size of the y_test dataset: (72, 1)


In [11]:
def evaluate_forecast(y_test_inverse, yhat_inverse):
    mse_ = keras.metrics.MeanSquaredError()
    mae_ = keras.metrics.MeanAbsoluteError()
    rmse_ = keras.metrics.RootMeanSquaredError()
    mae = mae_(y_test_inverse,yhat_inverse)
    print('mae:', mae)
    mse = mse_(y_test_inverse,yhat_inverse)
    print('mse:', mse)
    rmse = rmse_(y_test_inverse,yhat_inverse)
    print('rmse:', rmse)
    return mae, mse, rmse

In [10]:
nb_features = len(X_train_impute_df.columns)
output_activation = "linear"

## Simple Model Hyper-parameter Tuning
This model is just made of a single LSTM layer

In [12]:
class Model1(keras_tuner.HyperModel):
    def build(self, hp):
        model = Sequential()
        hp_units = hp.Int("units", min_value=20, max_value=300, step=20)
        hp_activ = hp.Choice("activation", values=["relu", "tanh", "sigmoid"])
        model.add(LSTM(units=hp_units, activation=hp_activ, input_shape=(7, nb_features)))
        model.add(Dense(1, activation=output_activation))

        hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        model.compile(loss="mse", optimizer=Adagrad(learning_rate=hp_learning_rate), metrics=[keras.metrics.RootMeanSquaredError()])

        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            validation_split=hp.Float("validation_split", min_value=0.05, max_value=0.2, step=0.05),
            batch_size=hp.Int("batch_size", min_value=32, max_value=192, step=32),
            #epochs=hp.Int("epochs", min_value=50, max_value=400, step=50),
            **kwargs,
        )

In [13]:
stop_early = tensorflow.keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', patience=7)
tuner = keras_tuner.Hyperband(Model1(),
                              objective=keras_tuner.Objective("val_root_mean_squared_error", direction="min"),
                              max_epochs=100000,
                              factor=1000,
                              overwrite=True,
                              directory="keras_tuner",
                              project_name="keras_tuner")

In [ ]:
tuner.search(X_train, y_train, epochs=100, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.
lstm_units: {best_hps.get('units')}
lstm_activation: {best_hps.get('activation')}
learning_rate: {best_hps.get('learning_rate')}
batch_size: {best_hps.get('batch_size')}
validation_split: {best_hps.get('validation_split')}
""")

Trial 561 Complete [00h 00m 34s]
val_root_mean_squared_error: 0.16563788056373596

Best val_root_mean_squared_error So Far: 0.08208870887756348
Total elapsed time: 03h 55m 49s

Search: Running Trial #562

Value             |Best Value So Far |Hyperparameter
160               |300               |units
sigmoid           |tanh              |activation
0.001             |0.01              |learning_rate
0.15              |0.15              |validation_split
192               |32                |batch_size
100               |100               |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/100
2/2 [==============================] - 2s 409ms/step - loss: 1.3211 - root_mean_squared_error: 1.1494 - val_loss: 1.3352 - val_root_mean_squared_error: 1.1555
Epoch 2/100
2/2 [==============================] - 0s 228ms/step - loss: 1.1088 - root_mean_squared_error: 1.0

## Simple Model Evaluation
The results of the evaluation of the tuned model compared to the test set are storred in Neptune

In [ ]:
m1_hyper_parameters = {
    "random_seed": RANDOM_SEED,
    "test_size": test_size,
    "validation_split": 0.1,
    "learning_rate": 0.01,
    "batch_size": 32,
    "epochs": 200,
    "lstm_units": 40,
    "lstm_activation": "sigmoid",
    "output_activation": output_activation,
    "nb_features": nb_features,
    "optimizer": "Adam"
}

m1_optimizer = {
    "RMSprop": RMSprop(learning_rate=m1_hyper_parameters["learning_rate"]),
    "Adam": Adam(learning_rate=m1_hyper_parameters["learning_rate"]),
    "Adamax": Adamax(learning_rate=m1_hyper_parameters["learning_rate"]),
    "Adagrad": Adagrad(learning_rate=m1_hyper_parameters["learning_rate"])
}

In [33]:
model1 = Sequential()
model1.add(LSTM(m1_hyper_parameters["lstm_units"], activation=m1_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model1.add(Dense(1, activation=m1_hyper_parameters["output_activation"]))
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 70)                42560     
                                                                 
 dense_2 (Dense)             (None, 1)                 71        
                                                                 
Total params: 42,631
Trainable params: 42,631
Non-trainable params: 0
_________________________________________________________________


In [91]:
# Start experiment
run = neptune.init(
    project="milestone2-california-water-shortage/deeplearning-lstm",
    api_token=neptune_key,
    name="Basic Model",
    tags=["WithDetailedWellCounts", "UnidirectionalLSTM"]
)
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
run['hyper-parameters'] = m1_hyper_parameters

model1.compile(loss="mse", optimizer=m1_optimizer[m1_hyper_parameters["optimizer"]], metrics=[keras.metrics.RootMeanSquaredError()])
model1.fit(X_train, y_train,
                     validation_split=m1_hyper_parameters["validation_split"],
                     batch_size=m1_hyper_parameters["batch_size"],
                     epochs=m1_hyper_parameters["epochs"],
                     shuffle=True,
                     callbacks=[neptune_cbk])
yhat = model1.predict(X_test, verbose=0)
yhat_inverse = scaler.inverse_transform(yhat)
y_test_inverse = scaler.inverse_transform(y_test)
mae, mse, rmse = evaluate_forecast(y_test_inverse, yhat_inverse)
run["eval/mae"] = mae
run["eval/mse"] = mse
run["eval/rmse"] = rmse
run.stop()

https://app.neptune.ai/milestone2-california-water-shortage/deeplearning-lstm/e/DEEPLSTM-87
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/200
12/12 [==============================] - 1s 27ms/step - loss: 0.2526 - val_loss: 0.0368
Epoch 2/200
12/12 [==============================] - 0s 9ms/step - loss: 0.0431 - val_loss: 0.0152
Epoch 3/200
12/12 [==============================] - 0s 8ms/step - loss: 0.0233 - val_loss: 0.0116
Epoch 4/200
12/12 [==============================] - 0s 9ms/step - loss: 0.0175 - val_loss: 0.0121
Epoch 5/200
12/12 [==============================] - 0s 8ms/step - loss: 0.0148 - val_loss: 0.0084
Epoch 6/200
12/12 [==============================] - 0s 9ms/step - loss: 0.0129 - val_loss: 0.0082
Epoch 7/200
12/12 [==============================] - 0s 9ms/step - loss: 0.0112 - val_loss: 0.0

## Model2 Hyper-parameter tuning
This model is made of
* a simple or bidirectional LSTM layer
* a Dense unit
* a Dropout Unit

In [14]:
class Model2(keras_tuner.HyperModel):
    def build(self, hp):
        model = Sequential()
        lstm_units = hp.Int("lstm_units", min_value=20, max_value=300, step=20)
        lstm_activ = hp.Choice("lstm_activation", values=["relu", "tanh", "sigmoid"])
        model.add(LSTM(units=lstm_units, activation=lstm_activ, input_shape=(7, nb_features)))
        dense_units = hp.Int("dense_units", min_value=11, max_value=101, step=10)
        dense_activation = hp.Choice("dense_activation", values=["relu", "tanh", "sigmoid"])
        model.add(Dense(dense_units, activation=dense_activation))
        hp_dropout = hp.Float("dropout_rate", min_value=0.05, max_value=0.25, step=0.05)
        model.add(Dropout(hp_dropout))
        model.add(Dense(1, activation=output_activation))

        hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        model.compile(loss="mse", optimizer=RMSprop(learning_rate=hp_learning_rate), metrics=[keras.metrics.RootMeanSquaredError()])

        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            validation_split=hp.Float("validation_split", min_value=0.05, max_value=0.2, step=0.05),
            batch_size=hp.Int("batch_size", min_value=32, max_value=192, step=32),
            #epochs=hp.Int("epochs", min_value=50, max_value=400, step=50),
            **kwargs,
        )

In [15]:
stop_early = tensorflow.keras.callbacks.EarlyStopping(monitor="val_root_mean_squared_error", patience=7)
tuner = keras_tuner.Hyperband(Model2(),
                              objective=keras_tuner.Objective("val_root_mean_squared_error", direction="min"),
                              max_epochs=100000,
                              factor=1000,
                              overwrite=True,
                              directory="keras_tuner",
                              project_name="keras_tuner")

In [16]:
tuner.search(X_train, y_train, epochs=100, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.
lstm_units: {best_hps.get('lstm_units')}
lstm_activation: {best_hps.get('lstm_activation')}
dense_units: {best_hps.get('dense_units')}
dense_activation: {best_hps.get('dense_activation')}
dropout_rate: {best_hps.get('dropout_rate')}
learning_rate: {best_hps.get('learning_rate')}
batch_size: {best_hps.get('batch_size')}
validation_split: {best_hps.get('validation_split')}
""")

Trial 1505 Complete [00h 00m 26s]
val_root_mean_squared_error: 0.11756662279367447

Best val_root_mean_squared_error So Far: 0.0819995179772377
Total elapsed time: 03h 54m 36s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete.
lstm_units: 240
lstm_activation: tanh
dense_units: 91
dense_activation: tanh
dropout_rate: 0.2
learning_rate: 0.001
batch_size: 128
validation_split: 0.15000000000000002



## Model 2 Evaluation

In [29]:
m2_hyper_parameters = {
    "random_seed": RANDOM_SEED,
    "test_size": test_size,
    "validation_split": 0.15,
    "learning_rate": 0.001,
    "batch_size": 128,
    "epochs": 200,
    "lstm_units": 240,
    "lstm_activation": "tanh",
    "dense_units": 91,
    "dense_activation": "tanh",
    "dropout": 0.2,
    "output_activation": "linear",
    "nb_features": nb_features,
    "optimizer": "RMSprop"
}

m2_optimizer = {
    "RMSprop": RMSprop(learning_rate=m2_hyper_parameters["learning_rate"]),
    "Adam": Adam(learning_rate=m2_hyper_parameters["learning_rate"]),
    "Adamax": Adamax(learning_rate=m2_hyper_parameters["learning_rate"]),
    "Adagrad": Adagrad(learning_rate=m2_hyper_parameters["learning_rate"])
}

model2 = Sequential()
model2.add(Bidirectional(LSTM(m2_hyper_parameters["lstm_units"], activation=m2_hyper_parameters["lstm_activation"]), input_shape=(7, nb_features)))
model2.add(Dense(m2_hyper_parameters["dense_units"], activation=m2_hyper_parameters["dense_activation"]))
model2.add(Dropout(m2_hyper_parameters["dropout"]))
model2.add(Dense(1, activation=m2_hyper_parameters["output_activation"]))
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 480)              618240    
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 91)                43771     
                                                                 
 dropout_1 (Dropout)         (None, 91)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 92        
                                                                 
Total params: 662,103
Trainable params: 662,103
Non-trainable params: 0
_________________________________________________________________


In [30]:
run = neptune.init(
    project="milestone2-california-water-shortage/deeplearning-lstm",
    api_token=neptune_key,
    name="Advanced Model 1",
    tags=["WithDetailedWellCounts", "BidirectionalLSTM"]
)
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
run['hyper-parameters'] = m2_hyper_parameters

model2.compile(loss="mse", optimizer=m2_optimizer[m2_hyper_parameters["optimizer"]], metrics=[keras.metrics.RootMeanSquaredError()])
model2.fit(X_train, y_train,
                     validation_split=m2_hyper_parameters["validation_split"],
                     batch_size=m2_hyper_parameters["batch_size"],
                     epochs=m2_hyper_parameters["epochs"],
                     shuffle=True,
                     callbacks=[neptune_cbk])
yhat = model2.predict(X_test, verbose=0)
yhat_inverse = scaler.inverse_transform(yhat)
y_test_inverse = scaler.inverse_transform(y_test)
evaluate_forecast(y_test_inverse, yhat_inverse)
mae, mse, rmse = evaluate_forecast(y_test_inverse, yhat_inverse)
run["eval/mae"] = mae
run["eval/mse"] = mse
run["eval/rmse"] = rmse
run.stop()

https://app.neptune.ai/milestone2-california-water-shortage/deeplearning-lstm/e/DEEPLSTM-140


Info (NVML): Not Supported. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/200
3/3 [==============================] - 4s 375ms/step - loss: 0.4868 - root_mean_squared_error: 0.6977 - val_loss: 0.0162 - val_root_mean_squared_error: 0.1272
Epoch 2/200
3/3 [==============================] - 0s 63ms/step - loss: 0.0140 - root_mean_squared_error: 0.1182 - val_loss: 0.0155 - val_root_mean_squared_error: 0.1243
Epoch 3/200
3/3 [==============================] - 0s 67ms/step - loss: 0.0123 - root_mean_squared_error: 0.1107 - val_loss: 0.0148 - val_root_mean_squared_error: 0.1216
Epoch 4/200
3/3 [==============================] - 0s 64ms/step - loss: 0.0098 - root_mean_squared_error: 0.0991 - val_loss: 0.0130 - val_root_mean_squared_error: 0.1140
Epoch 5/200
3/3 [==============================] - 0s 66ms/step - loss: 0.0076 - root_mean_squared_er

## Model3 Hyper-parameter tuning
* a simple or bidirectional encoding LSTM layer
* a simple or bidirectional decoding LSTM layer
* a Dense unit
* a Dropout Unit

In [ ]:
class Model3(keras_tuner.HyperModel):
    def build(self, hp):
        model = Sequential()
        lstm_units = hp.Int("lstm_units", min_value=20, max_value=300, step=20)
        lstm_activ = hp.Choice("lstm_activation", values=["relu", "tanh", "sigmoid"])
        model.add(LSTM(units=lstm_units, activation=lstm_activ, input_shape=(7, nb_features)))
        model3.add(RepeatVector(1))
        lstm_units_2 = hp.Int("lstm_units", min_value=20, max_value=300, step=20)
        model.add(LSTM(units=lstm_units_2, activation=lstm_activ, return_sequences=True))
        dense_units = hp.Int("2nd_lstm_units", min_value=11, max_value=101, step=10)
        dense_activation = hp.Choice("dense_activation", values=["relu", "tanh", "sigmoid"])
        model.add(TimeDistributed(Dense(dense_units, activation=dense_activation)))
        hp_dropout = hp.Float("dropout_rate", min_value=0.05, max_value=0.25, step=0.05)
        model.add(Dropout(hp_dropout))
        model.add(Dense(1, activation=output_activation))

        hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        model.compile(loss="mse", optimizer=RMSprop(learning_rate=hp_learning_rate), metrics=[keras.metrics.RootMeanSquaredError()])

        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            validation_split=hp.Float("validation_split", min_value=0.05, max_value=0.2, step=0.05),
            batch_size=hp.Int("batch_size", min_value=32, max_value=192, step=32),
            #epochs=hp.Int("epochs", min_value=50, max_value=400, step=50),
            **kwargs,
        )

In [ ]:
stop_early = tensorflow.keras.callbacks.EarlyStopping(monitor="val_root_mean_squared_error", patience=7)
tuner = keras_tuner.Hyperband(Model3(),
                              objective=keras_tuner.Objective("val_root_mean_squared_error", direction="min"),
                              max_epochs=100000,
                              factor=1000,
                              overwrite=True,
                              directory="keras_tuner",
                              project_name="keras_tuner")

In [ ]:
tuner.search(X_train, y_train, epochs=100, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.
lstm_units: {best_hps.get('lstm_units')}
2nd_lstm_units: {best_hps.get('2nd_lstm_units')}
lstm_activation: {best_hps.get('lstm_activation')}
dense_units: {best_hps.get('dense_units')}
dense_activation: {best_hps.get('dense_activation')}
dropout_rate: {best_hps.get('dropout_rate')}
learning_rate: {best_hps.get('learning_rate')}
batch_size: {best_hps.get('batch_size')}
validation_split: {best_hps.get('validation_split')}
""")

## Model 3 Evaluation

In [64]:
m3_hyper_parameters = {
    "random_seed": RANDOM_SEED,
    "test_size": test_size,
    "validation_split": 0.1,
    "learning_rate": 0.01,
    "batch_size": 64,
    "epochs": 200,
    "lstm_units": 200,
    "2nd_lstm_units": 100,
    "lstm_activation": "sigmoid",
    "dense_units": 81,
    "dense_activation": "tanh",
    "dropout": 0.2,
    "output_activation": "linear",
    "nb_features": nb_features,
    "optimizer": "RMSprop"
}

m3_optimizer = {
    "RMSprop": RMSprop(learning_rate=m3_hyper_parameters["learning_rate"]),
    "Adam": Adam(learning_rate=m3_hyper_parameters["learning_rate"]),
    "Adamax": Adamax(learning_rate=m3_hyper_parameters["learning_rate"]),
    "Adagrad": Adagrad(learning_rate=m3_hyper_parameters["learning_rate"])
}

model3 = Sequential()
model3.add(LSTM(m3_hyper_parameters["lstm_units"], activation=m3_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model3.add(RepeatVector(1))
model3.add(LSTM(m3_hyper_parameters["2nd_lstm_units"], activation=m3_hyper_parameters["lstm_activation"], return_sequences=True))
model3.add(TimeDistributed(Dense(m3_hyper_parameters["dense_units"], activation=m3_hyper_parameters["dense_activation"])))
model3.add(Dropout(m3_hyper_parameters["dropout"]))
model3.add(Dense(1, activation=m3_hyper_parameters["output_activation"]))
model3.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 100)               72800     
                                                                 
 repeat_vector_9 (RepeatVect  (None, 1, 100)           0         
 or)                                                             
                                                                 
 lstm_19 (LSTM)              (None, 1, 100)            80400     
                                                                 
 time_distributed_9 (TimeDis  (None, 1, 81)            8181      
 tributed)                                                       
                                                                 
 dropout_9 (Dropout)         (None, 1, 81)             0         
                                                                 
 dense_19 (Dense)            (None, 1, 1)             

In [65]:
run = neptune.init(
    project="milestone2-california-water-shortage/deeplearning-lstm",
    api_token=neptune_key,
    name="Advanced Model 4",
    tags=["WithDetailedWellCounts", "OneUnidirectionalLSTM", "OneRepeatVector", "OneUnidirectionalLSTM", "TimeDistributedDense", "Dropout"]
)
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
run['hyper-parameters'] = m3_hyper_parameters

model3.compile(loss="mse", optimizer=m3_optimizer[m3_hyper_parameters["optimizer"]], metrics=[keras.metrics.RootMeanSquaredError()])
model3.fit(X_train, y_train_3d,
                     validation_split=m3_hyper_parameters["validation_split"],
                     batch_size=m3_hyper_parameters["batch_size"],
                     epochs=m3_hyper_parameters["epochs"],
                     shuffle=False,
                     callbacks=[neptune_cbk])
yhat = model3.predict(X_test, verbose=0)
yhat_inverse = scaler.inverse_transform(yhat.squeeze(2))
y_test_inverse = scaler.inverse_transform(y_test)
evaluate_forecast(y_test_inverse, yhat_inverse)
mae, mse, rmse = evaluate_forecast(y_test_inverse, yhat_inverse)
run["eval/mae"] = mae
run["eval/mse"] = mse
run["eval/rmse"] = rmse
run.stop()

https://app.neptune.ai/milestone2-california-water-shortage/deeplearning-lstm/e/DEEPLSTM-80
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/200
2/2 [==============================] - 4s 750ms/step - loss: 0.0954 - val_loss: 0.0624
Epoch 2/200
2/2 [==============================] - 0s 66ms/step - loss: 0.0453 - val_loss: 0.0285
Epoch 3/200
2/2 [==============================] - 0s 67ms/step - loss: 0.0536 - val_loss: 0.0317
Epoch 4/200
2/2 [==============================] - 0s 68ms/step - loss: 0.0327 - val_loss: 0.0286
Epoch 5/200
2/2 [==============================] - 0s 69ms/step - loss: 0.0293 - val_loss: 0.0291
Epoch 6/200
2/2 [==============================] - 0s 69ms/step - loss: 0.0234 - val_loss: 0.0296
Epoch 7/200
2/2 [==============================] - 0s 67ms/step - loss: 0.0180 - val_loss: 0.0287
Epo